TP Groupe - Noé Duhamel, Guillaume Gatille & Nathan Stooss

In [ ]:
import os
import re
from collections import Counter, defaultdict
from pprint import pprint
import string

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.util import ngrams
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.cluster import KMeans

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import gensim
from gensim import corpora
from gensim.models import CoherenceModel, LdaModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

In [ ]:
df_path = "bible.csv"
df = pd.read_csv(df_path)
print("Rows: " + format(df.shape[0]))
print("Columns: " + format(df.shape[1]))

In [ ]:
df.head()

Définissons la variable qui divisera l'ensemble des données en Nouveau et Ancien Testament.

In [ ]:
df['t'] = df['t'].astype('str')
df.loc[df['b'] <= 39, 'testament'] = 'Old'
df.loc[df['b'] > 39, 'testament'] = 'New'
df

In [ ]:
# Stats descriptives sur le dataframe
df.rename(columns={
    'b': 'book_id', 
    'c': 'chapter_id',
    'v': 'verse_id',
    't': 'text'
    }, inplace=True)

df

## Nettoyage

In [ ]:
# Obtention de la liste des stopwords à supprimer (plus performante que la librairie de NLTK par défaut)
words_to_delete = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
                   "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
                   'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 
                   'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
                   'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 
                   'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 
                   'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 
                   'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
                   'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 
                   'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 
                   'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 
                   'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', 
                   "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 
                   'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
                    , 'unto', 'thou', 'thee', 'ye', 'him', 'upon', 'say', 'me', 'hath', 'also',"shouldn't", "wasn't", "weren't", 
                    "won't", "wouldn't", "unto", "thou", "thee", "ye", "him", "upon", "say", "me", "hath", "also", "ye"
                  ]

In [ ]:
# Retirer les mots spécifiés du texte
df['text_cleaned'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in words_to_delete]))

# Retirer les chiffres
df['text_cleaned'] = df['text_cleaned'].str.replace(r'\d+', '')

# Retirer la ponctuation
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Retirer les espaces en trop
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

# Convertir text_cleaned en minuscule
df['text_cleaned'] = df['text_cleaned'].str.lower()

# Lemmatisation
lemmatizer = WordNetLemmatizer()
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

# Export en CSV pour éviter le retraitement sur des ordinateurs moins performants
df.to_csv('bible-cleaned.csv')

In [ ]:
df.head()

In [ ]:
# Test d'efficacité du nettoyage — Impression de la première ligne
# Cela permet de comparer les différentes étapes de nettoyage
print('Cleaned text: \n' + str(df['text'][0]))
print('Cleaned text: \n' + str(df['text_cleaned'][0]))

# Data Analysis

In [ ]:
df.info()

In [ ]:
columns_to_convert = ['id', 'book_id', 'chapter_id', 'verse_id']
df[columns_to_convert] = df[columns_to_convert].astype('int16')
print(df.dtypes)

In [ ]:
columns_to_convert = ['text', 'testament', 'text_cleaned']
df[columns_to_convert] = df[columns_to_convert].astype('string')
print(df.dtypes)


In [ ]:
# Ajout des colonnes contenant des stats descriptives
df['word_count'] = df['text_cleaned'].apply(lambda x: len(x.split()))
df['unique_word_count'] = df['text_cleaned'].apply(lambda x: len(set(x.split())))
df['sentence_count'] = df['text_cleaned'].apply(lambda x: len(sent_tokenize(x)))
df['avg_word_length'] = df['text_cleaned'].apply(lambda x: np.mean([len(word) for word in x.split()])).round(2)
df

Avec cet ensemble de données donné, on peux appliquer une analyse complète. Tout d'abord, on commence par un sujet intéressant : comment la longueur des versets évolue à travers les livres de la Bible. 
Pourquoi cette information peut-elle être utile ? Elle nous permet d'estimer approximativement quand les livres ont été écrits et la longueur des versets offre beaucoup de connaissances sur la culture à ce moment.

In [ ]:
verses_column = 'verse_id'
words_column = 'word_count'

# Ajout de couleur
color_1 = plt.cm.Blues(np.linspace(0.6, 1, 66))
color_2 = plt.cm.Purples(np.linspace(0.6, 1, 66))

# Regroupement par 'book_id' représentant les livres de la Bible.
words_verses = df.groupby('book_id').agg({verses_column: 'count', words_column: 'sum'}).sort_values(by=verses_column, ascending=False)
data1 = words_verses[verses_column]
data2 = words_verses[words_column]

plt.figure(figsize=(16, 8))
x = np.arange(66)
ax1 = plt.subplot(1, 1, 1)
w = 0.3

color = color_1
plt.title('Nombre de mots vs nombre de versets')
plt.xticks(x + w / 2, data1.index, rotation=-90)
ax1.set_xlabel('Livres de la Bible')
ax1.set_ylabel('Nombre de versets')
ax1.bar(x, data1.values, color=color_1, width=w, align='center')

ax2 = ax1.twinx()

color = color_2
ax2.set_ylabel('Nombre de mots')
ax2.bar(x + w, data2, color=color_2, width=w, align='center')

plt.show()


En visualisant l'évolution du nombre total de mots par livre, on peux  obtenir des informations sur la longueur relative des livres. Certains livres peuvent être plus longs que d'autres, ce qui peut refléter la complexité de leur contenu.

In [ ]:
# # Calcul du nombre total de mots par livre
# total_words_by_book = df.groupby('book_id')['word_count'].sum().reset_index()

# # Plotting
# plt.figure(figsize=(15, 6))
# sns.lineplot(x='book_id', y='word_count', data=total_words_by_book, marker='o', color='green')
# plt.title('Evolution of the Number of Words in Books')
# plt.xlabel('Book ID')
# plt.ylabel('Total Word Count')
# plt.show()

In [ ]:
# Group the dataframe by book_id and calculate the sum of occurrences for each word
verses_counts = df.groupby('book_id')['verse_id'].count()
words_counts = df.groupby('book_id')['word_count'].sum()

# Create a bar plot
plt.figure(figsize=(12, 6))
plt.bar(verses_counts.index, words_counts, width=0.8)
plt.xlabel('Book ID')
plt.ylabel('Number of Words')
plt.title('Number of Words per Books')
plt.show()

En analysant graphiquement les données, on peut rapidement identifier les livres où les références à "God" et "Jesus" sont plus fréquentes, ainsi que les tendances générales au fil des différents IDs de livre. Ici Jésus apparait dans le New testament logiquement.

In [ ]:
# Regroupement des données par numéro de livre et calcul de la somme des occurrences pour chaque mot
god_counts = df.groupby('book_id')['text_cleaned'].apply(lambda x: x.explode().str.count('god').sum())
jesus_counts = df.groupby('book_id')['text_cleaned'].apply(lambda x: x.explode().str.count('jesus').sum())

# Création d'un graphique
plt.figure(figsize=(10, 6))
plt.plot(god_counts.index, god_counts.values, label='God')
plt.plot(jesus_counts.index, jesus_counts.values, label='Jesus')
plt.xlabel('Book ID')
plt.ylabel('Nombre d\'occurences')
plt.title('Evolution des occurences des mots (God and Jesus)')
plt.legend()
plt.show()

Ce graphique facilite  l'identification de tendances générales dans l'utilisation de ces termes clés lié au pouvoir et facilitant ainsi une analyse thématique eventuel.

In [ ]:
# Regroupement des données par numéro de livre et calcul de la somme des occurrences pour chaque mot
lord_counts = df.groupby('book_id')['text_cleaned'].apply(lambda x: x.explode().str.count('lord').sum())
god_counts = df.groupby('book_id')['text_cleaned'].apply(lambda x: x.explode().str.count('god').sum())
king_counts = df.groupby('book_id')['text_cleaned'].apply(lambda x: x.explode().str.count('king').sum())

# Création d'un graphique
plt.figure(figsize=(10, 6))
plt.plot(lord_counts.index, lord_counts.values, label='Lord')
plt.plot(god_counts.index, god_counts.values, label='God')
plt.plot(king_counts.index, king_counts.values, label='King')
plt.xlabel('Book ID')
plt.ylabel('Number of Occurrences')
plt.title('Evolution of Word Occurrences (Lord, God, King)')
plt.legend()
plt.show()


In [ ]:
La représentation graphique des termes "Holy," "Son," et "Father" dans l'Ancien Testament et le NOUVEAU est utile car elle offre une vision  des thèmes théologiques ET permet une compréhension  rapide du vocabulaire utilisé.

# Group the dataframe by book_id and calculate the sum of occurrences for each word
holy_spirit_counts = df.groupby('book_id')['text_cleaned'].apply(lambda x: x.explode().str.count('holy').sum())
son_counts = df.groupby('book_id')['text_cleaned'].apply(lambda x: x.explode().str.count('son').sum())
father_counts = df.groupby('book_id')['text_cleaned'].apply(lambda x: x.explode().str.count('father').sum())

# Create a line plot
plt.figure(figsize=(10, 6))
plt.plot(holy_spirit_counts.index, holy_spirit_counts.values, label='Holy')
plt.plot(son_counts.index, son_counts.values, label='Son')
plt.plot(father_counts.index, father_counts.values, label='Father')
plt.xlabel('Book ID')
plt.ylabel('Number of Occurrences')
plt.title('Evolution of Word Occurrences (Holy, Son, Father)')
plt.legend()
plt.show()

Visualisation en nuage de mots de ceux les plus utilisés entre ancien et nouveau testament

In [ ]:
# Filtrage la base de données pour obtenir le texte de l'Ancien Testament
old_testament_text = ' '.join(df[df['testament'] == 'Old']['text'])

# Générer un nuage de mots pour l'Ancien Testament
wordcloud_old = WordCloud(width=800, height=400, max_font_size=150).generate(old_testament_text)

# Filtrer le dataframe pour obtenir le texte du Nouveau Testament
new_testament_text = ' '.join(df[df['testament'] == 'New']['text'])

# Générer un nuage de mots pour le Nouveau Testament
wordcloud_new = WordCloud(width=800, height=400, max_font_size=150).generate(new_testament_text)

# Plot the word clouds
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_old, interpolation='bilinear')
plt.title('Nuage de mots - Ancien Testament')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(wordcloud_new, interpolation='bilinear')
plt.title('Nuage de mots - Nouveau Testament')
plt.axis('off')

plt.show()



## N-gramm / Bi-gramm / Tri-gramm

In [ ]:
# Fonction pour obtenir les n-grammes les plus fréquents
def get_most_frequent_ngrams(text, n, top_k):
    # Tokeniser le texte en mots
    words = text.split()
    
    # Générer les n-grammes
    ngrams_list = list(ngrams(words, n))
    
    # Compter la fréquence de chaque n-gramme
    ngrams_freq = Counter(ngrams_list)
    
    # Obtenir les k n-grammes les plus fréquents
    top_ngrams = ngrams_freq.most_common(top_k)
    
    return top_ngrams

# Obtenir les bigrammes les plus fréquents dans l'Ancien Testament
old_testament_bigrams = get_most_frequent_ngrams(old_testament_text, 2, 10)

# Obtenir les bigrammes les plus fréquents dans le Nouveau Testament
new_testament_bigrams = get_most_frequent_ngrams(new_testament_text, 2, 10)

# Tracer les graphiques à barres pour les bigrammes de l'Ancien et du Nouveau Testament
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Bigrammes de l'Ancien Testament
axes[0].barh([str(bigram) for bigram, frequency in old_testament_bigrams], [frequency for bigram, frequency in old_testament_bigrams], color='blue')
axes[0].set_title('Top 10 des bigrammes dans l\'Ancien Testament')
axes[0].set_xlabel('Fréquence')

# Bigrammes du Nouveau Testament
axes[1].barh([str(bigram) for bigram, frequency in new_testament_bigrams], [frequency for bigram, frequency in new_testament_bigrams], color='green')
axes[1].set_title('Top 10 des bigrammes dans le Nouveau Testament')
axes[1].set_xlabel('Fréquence')

plt.tight_layout()
plt.show()


In [ ]:
# Fonction pour obtenir les n-grammes les plus fréquents
def get_most_frequent_ngrams(text, n, top_k):
    # Tokeniser le texte en mots
    words = text.split()
    
    # Générer les n-grammes
    ngrams_list = list(ngrams(words, n))
    
    # Compter la fréquence de chaque n-gramme
    ngrams_freq = Counter(ngrams_list)
    
    # Obtenir les k n-grammes les plus fréquents
    top_ngrams = ngrams_freq.most_common(top_k)
    
    return top_ngrams

# Obtenir les trigrammes les plus fréquents dans l'Ancien Testament
old_testament_trigrams = get_most_frequent_ngrams(old_testament_text, 3, 10)

# Obtenir les trigrammes les plus fréquents dans le Nouveau Testament
new_testament_trigrams = get_most_frequent_ngrams(new_testament_text, 3, 10)

# Trier les trigrammes par fréquence en ordre décroissant
old_testament_trigrams.sort(key=lambda x: x[1], reverse=True)
new_testament_trigrams.sort(key=lambda x: x[1], reverse=True)

# Tracer les graphiques à barres pour les trigrammes de l'Ancien et du Nouveau Testament
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Trigrammes de l'Ancien Testament
axes[0].barh([str(trigram) for trigram, frequency in old_testament_trigrams], [frequency for trigram, frequency in old_testament_trigrams], color='blue')
axes[0].set_title('Top 10 des trigrammes dans l\'Ancien Testament')
axes[0].set_xlabel('Fréquence')

# Trigrammes du Nouveau Testament
axes[1].barh([str(trigram) for trigram, frequency in new_testament_trigrams], [frequency for trigram, frequency in new_testament_trigrams], color='green')
axes[1].set_title('Top 10 des trigrammes dans le Nouveau Testament')
axes[1].set_xlabel('Fréquence')

plt.tight_layout()
plt.show()


## BOW - Corpus

In [ ]:
# Utilisation de CountVectorizer pour obtenir les caractéristiques du BOW
cv = CountVectorizer()
cv_matrix = cv.fit_transform(df['text_cleaned'])

# Mots uniques dans le corpus
vocab = cv.get_feature_names_out()

# Affichage des vecteurs de caractéristiques des documents dans un DataFrame
bible_df_feature_vectors = pd.DataFrame(cv_matrix.toarray(), columns=vocab)

# Affichage du DataFrame
bible_df_feature_vectors

## TF-IDF

In [ ]:
# Aggrégation des versets par livre pour effecter une analyse par livre

books = {}
for verse in range(len(df)):
    current_book = df['book_id'].iloc[verse]
    if(current_book in books):
        books[current_book] = str(books[current_book]) + str(df['text_cleaned'].iloc[verse]) + ' '
    else: 
        books[current_book] = str(df['text_cleaned'].iloc[verse]) + ' '

books_list = list(books.values())
books_df = pd.DataFrame(books_list, columns=['corpus'])

del books_list

In [ ]:
# Création d'un objet TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit du TfidfVectorizer sur la colonne 'text_cleaned' des livres échantillonnés
tfidf_matrix = tfidf_vectorizer.fit_transform(books_df['corpus'])

# Vocabulaire
vocab = tfidf_vectorizer.get_feature_names_out()

# Création d'une liste pour stocker les 3 principaux termes TF-IDF pour chaque livre
top_tfidf_terms_list = []

# Pour chaque livre échantillonné, obtention des  3 principaux termes TF-IDF
for idx, book in enumerate(books_df['corpus']):
    tfidf_scores = tfidf_matrix[idx, :].toarray().flatten()
    
    # Création d'un dictionnaire pour stocker le terme et son score TF-IDF
    tfidf_dict = {term: round(score, 2) for term, score in zip(vocab, tfidf_scores)}
    
    # Tri des termes par score TF-IDF, et obtention des trois premiers
    sorted_tfidf_terms = sorted(tfidf_dict.items(), key=lambda x: x[1], reverse=True)[:3]
    
    # Ajout des résultats à la liste
    top_tfidf_terms_list.append({'Book': idx, 'Top 3 Terms': sorted_tfidf_terms})

# Création d'un DataFrame à partir de la liste
top_tfidf_terms_df = pd.DataFrame(top_tfidf_terms_list)

top_tfidf_terms_df


# LDA 

Le dictionnaire est un mappage entre les mots et leurs identifiants entiers,
Le corpus est une liste de documents représentés sous forme d'un BoW.

In [ ]:
documents = df["text_cleaned"].apply(lambda x: x.split(' '))

# Création d'un dictionnaire
id2word = corpora.Dictionary(documents)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in documents]

In [ ]:
# Initialisation des variables pour le meilleur modèle
best_coherence = -1
best_lda_model = None
best_num_topics = 0

# Esssai de différents nombres de topics
for num_topics in range(2, 10):
    # Construction du modèle LDA
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42, passes=10, alpha="auto", per_word_topics=True)

    # Calcul du score de cohérence
    coherence_model_lda = CoherenceModel(model=lda_model, texts=documents, dictionary=id2word, coherence="c_v")
    coherence_lda = coherence_model_lda.get_coherence()

    # Pour chaque nombre de topic, affichage du score de cohérence lié
    print(f"Nombre de topics: {num_topics}, score de cohérence: {coherence_lda}")

    # Stockage et mise à jour du meilleur modèle, si ce dernier est plus cohérent
    if coherence_lda > best_coherence:
        best_coherence = coherence_lda
        best_lda_model = lda_model
        best_num_topics = num_topics


In [ ]:
# Affichage du meilleur nombre de topics
print(f"\nBest Number of Topics: {best_num_topics}")

# Imprimer les mots-clés pour chaque sujet dans le meilleur modèle
pprint(best_lda_model.print_topics())

Nous avons décidé de calculer la cohérence pour différents nombres de sujets et à choisir le modèle qui donne la cohérence la plus élevée. Ici 3 est indiqué comme le meilleur nombres de Topics.

In [ ]:
# Afficher la visualisation du meilleur modèle
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(best_lda_model, corpus, id2word)
pyLDAvis.display(vis)

# Clustering

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_cleaned'])

In [ ]:
pca = PCA(n_components=2)
reduced_tfidf = pca.fit_transform(tfidf_matrix.toarray())
reduced_tfidf.shape

In [ ]:
scores = []  # ici on va stocker les scores
cluster_range = range(1, 10)  # et ici le nombre de clusters que l'on veut tester (de 1 à 10)
for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=100)
    kmeans.fit(reduced_tfidf)
    scores.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, scores, marker='o', linestyle='--')
plt.xlabel('Number of Clusters')
plt.ylabel('Within-Cluster Sum of Squares')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.grid(True)
plt.show()

In [ ]:
optimal_k = 3  # Replace 3 with the actual optimal number of clusters

kmeans = KMeans(n_clusters=optimal_k, random_state=100)
clusters = kmeans.fit_predict(reduced_tfidf)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(reduced_tfidf[:, 0], reduced_tfidf[:, 1], c=clusters, cmap='rainbow')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=200, c='black', marker='X', label='Centroids')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Clusters des livres de la Bible')
plt.legend()
plt.grid(True)
plt.show()

# Sentiment Analysis avec NLTK

In [ ]:
df

In [ ]:
# Initialiser l'analyseur d'intensité des sentiments
sia = SentimentIntensityAnalyzer()

# Définir une fonction pour calculer le score de sentiment pour un verset donné
def get_sentiment_score(verse):
    # Calculate sentiment score
    sentiment_score = sia.polarity_scores(verse)['compound']
    return sentiment_score

# Apply the sentiment analysis function to the 'text_cleaned' column in your DataFrame
df['sentiment_score'] = df['text_cleaned'].apply(get_sentiment_score)

# Categorize sentiment based on the sentiment score
df['sentiment'] = df['sentiment_score'].apply(lambda score: 'positive' if score > 0 else 'negative' if score < 0 else 'neutral')

df[['text_cleaned', 'sentiment_score', 'sentiment']]

In [ ]:
# Regrouper les données par book_id et sentiment
grouped_data = df.groupby(['book_id', 'sentiment']).size().unstack()

# Calculer la proportion de chaque sentiment par book_id
proportional_data = grouped_data.div(grouped_data.sum(axis=1), axis=0)
proportional_data.plot(figsize=(12, 6), kind='bar', stacked=True, width=0.8, color=['#FF4D4D', '#66CC66', '#4DA6FF'])